In [84]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

In [85]:
# Load the response.csv file and ensure columns are correctly named
df = pd.read_csv("responses.csv")

# Ensure the 'variant' column is of type string and no NaN values
df['variant'] = df['variant'].fillna('').astype(str)

# Shuffle the dataframe, keeping 'response' and 'variant' in sync
df = df.sample(frac=1).reset_index(drop=True)

# Add a new column to store feedback if not already present
if 'feedback' not in df.columns:
    df['feedback'] = pd.Series(dtype='float')

# Initialize response index
response_index = 0

In [86]:
from main import update_response


def on_button_clicked(b):
    global response_index
    # Convert thumbs up / down to 1 / 0
    user_feedback = 1 if b.description == "\U0001F44D" else 0
    
    # Check if the index is within the bounds
    if response_index < len(df):
        # Update the feedback column
        df.at[response_index, 'feedback'] = user_feedback
        print(f"Feedback recorded for index {response_index}: {user_feedback}")

        response_index += 1

        if response_index < len(df):
            update_response()
        else:
            # Save the feedback to a CSV file
            df.to_csv("results.csv", index=False)
            print("A/B testing completed. Here's the results:")
            print(df[['response', 'variant', 'feedback']])  # Print the DataFrame to verify
            if 'variant' in df.columns:
                summary_df = df.groupby('variant').agg(
                    count=('feedback', 'count'),
                    score=('feedback', 'mean')).reset_index()
                print(summary_df)
            else:
                print("No 'variant' column found to group results.")
    else:
        print("Error: Index out of bounds.")

In [87]:
from urllib import response


def update_response():
    global response_index
    if response_index < len(df):
        new_response = df.iloc[response_index]['response']
        prompt = "<h2>Please review the following response and provide your feedback:</h2> "
        response.value = f"{prompt}<p>{new_response}</p>"
        count_label.value = f"Response: {response_index + 1}/{len(df)}"
    else:
        response.value = "<p>No more responses.</p>"
        count_label.value = ""
        
# Create widgets
response = widgets.HTML()
count_label = widgets.Label()
thumbs_up_button = widgets.Button(description='\U0001F44D')
thumbs_down_button = widgets.Button(description='\U0001F44E')

# Connect buttons to callback
thumbs_up_button.on_click(on_button_clicked)
thumbs_down_button.on_click(on_button_clicked)

# Display widgets
button_box = widgets.HBox([thumbs_down_button, thumbs_up_button])
display(response, button_box, count_label)
update_response()

HTML(value='')

Label(value='')

Feedback recorded for index 0: 1
Feedback recorded for index 1: 1
Feedback recorded for index 2: 0
Feedback recorded for index 3: 0
Feedback recorded for index 4: 1
Feedback recorded for index 5: 1
Feedback recorded for index 6: 0
Feedback recorded for index 7: 1
Feedback recorded for index 8: 1
Feedback recorded for index 9: 1
A/B testing completed. Here's the results:
                                            response variant  feedback
0                  Would you purchase from us again?       A       1.0
1           How easy was it to navigate our website?       B       1.0
2  How would you rate the value for money of our ...       B       0.0
3  How satisfied are you with the range of produc...       B       0.0
4  How clear and accurate was the product descrip...       A       1.0
5            Did the product meet your expectations?       A       1.0
6                  Was the product delivery on time?       A       0.0
7  How likely are you to recommend our product to...       